In [1]:
import cv2
import Levenshtein
import optuna
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn.metrics
import ipywidgets
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm.auto import tqdm
tqdm.pandas()

/opt/miniconda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
%load_ext autoreload
%autoreload 2
from nncomp_molecule import constants

In [3]:
train_df = pd.read_pickle("/work/input/kfujikawa/bms-preprocess-v2/train.pkl").set_index("image_id")
train_df.head()

,InChI,image_path,InChI_length,w,h,w/h,w*h,is_flipped
image_id,,,,,,,,
4435736fd10b,InChI=1S/C65H110O6/c1-4-7-10-13-16-19-22-25-28...,/work/input/bms-molecular-translation/train/4/...,403,1472,788,1.868020,1159936,False
8c0e35ce3f1f,InChI=1S/C60H98O6/c1-4-7-10-13-16-19-22-25-27-...,/work/input/bms-molecular-translation/train/8/...,398,939,309,3.038835,290151,False
934593ad3cae,InChI=1S/C63H106O6/c1-4-7-10-13-16-19-22-25-27...,/work/input/bms-molecular-translation/train/9/...,397,826,661,1.249622,545986,False
c963808e309d,InChI=1S/C59H92O6/c1-4-7-10-13-16-19-22-25-28-...,/work/input/bms-molecular-translation/train/c/...,393,1268,883,1.436014,1119644,False
0aa425d5f5ac,InChI=1S/C62H106O6/c1-4-7-10-13-16-19-22-25-27...,/work/input/bms-molecular-translation/train/0/...,393,918,543,1.690608,498474,False


In [1]:
COLUMNS = [
    "image_id", "InChI", "normed_InChI", "InChI_GT", "is_valid", "normed_score", "levenshtein"
]
VALID_CSVs = [
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=4.csv",
    constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "valid_beam=4.csv",
    constants.OUTPUTDIR / "9005_1102+1105+1106" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "v30" / "valid_pred.csv",
]
valid_df = pd.concat([
    pd.read_csv(filename, usecols=COLUMNS).assign(model=filename.parent.name)
    for filename in tqdm(VALID_CSVs)
], ignore_index=True)
valid_df["image_path"] = valid_df.image_id.map(train_df.image_path)

NameError: name 'constants' is not defined

In [5]:
sort_keys = dict(
    image_id=True,
    is_valid=False,
    normed_score=True,
)
valid_ensembled_df = valid_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby(["image_id"]).first()
base_score = valid_ensembled_df.levenshtein.mean()
lower_score = valid_df.groupby("image_id").levenshtein.min().mean()
print(f"Levenshtein: {base_score}")
print(f"Levenshtein (lower): {lower_score}")

Levenshtein: 0.7485180617030843
Levenshtein (lower): 0.17039918488237307


In [8]:
def re_validate_inchi(normed_InChI):
    if "/q" in normed_InChI:
        return False
    if "/p" in normed_InChI:
        return False
    if "?" in normed_InChI:
        return False
    return True
    

is_valid = valid_df.normed_InChI.progress_apply(re_validate_inchi)
valid_df["is_valid"] = valid_df.is_valid.where(is_valid, False)
valid_df["normed_InChI"] = valid_df.normed_InChI.where(is_valid, valid_df.InChI)

In [16]:
valid_df

,image_id,InChI,InChI_GT,is_valid,normed_InChI,normed_score,levenshtein,model,image_path
0,a3c096ab64e7,InChI=1S/C56H90O6/c1-4-7-10-13-16-19-22-25-26-...,InChI=1S/C57H90O6/c1-4-7-10-13-16-19-22-25-26-...,False,InChI=1S/C56H90O6/c1-4-7-10-13-16-19-22-25-26-...,0.033260,94,1113_swin_large_bert_384,/work/input/bms-molecular-translation/train/a/...
1,01d9b7ce6ba1,InChI=1S/C61H104O6/c1-4-7-10-13-16-19-22-24-26...,InChI=1S/C62H106O6/c1-4-7-10-13-16-19-22-25-27...,True,InChI=1S/C61H104O6/c1-4-7-10-13-16-19-22-24-26...,0.013260,104,1113_swin_large_bert_384,/work/input/bms-molecular-translation/train/0/...
2,75c32fbd3779,InChI=1S/C72H135NO5/c1-3-5-7-9-11-13-15-17-19-...,InChI=1S/C69H127NO5/c1-3-5-7-9-11-13-15-17-18-...,False,InChI=1S/C72H135NO5/c1-3-5-7-9-11-13-15-17-19-...,0.035122,119,1113_swin_large_bert_384,/work/input/bms-molecular-translation/train/7/...
3,b440ab35d6a0,InChI=1S/C57H92O6/c1-4-7-10-13-16-19-22-25-27-...,InChI=1S/C56H90O6/c1-4-7-10-13-16-19-22-25-26-...,False,InChI=1S/C57H92O6/c1-4-7-10-13-16-19-22-25-27-...,0.032751,74,1113_swin_large_bert_384,/work/input/bms-molecular-translation/train/b/...
4,807b043f3c86,InChI=1S/C55H88O6/c1-4-7-10-13-16-19-22-25-27-...,InChI=1S/C56H88O6/c1-4-7-10-13-16-19-22-25-27-...,False,InChI=1S/C55H88O6/c1-4-7-10-13-16-19-22-25-27-...,0.027721,72,1113_swin_large_bert_384,/work/input/bms-molecular-translation/train/8/...
...,...,...,...,...,...,...,...,...,...
3878699,cbaa4844a4a4,"InChI=1S/CHF7NS/c2-1(3)9-10(4,5,6)7/b9-1-","InChI=1S/CF7NS/c2-1(3)9-10(4,5,6,7)8",False,"InChI=1S/CHF7NS/c2-1(3)9-10(4,5,6)7/b9-1-",0.132592,8,9006_1103+1106+1109,/work/input/bms-molecular-translation/train/c/...
3878700,ff46160b9517,"InChI=1S/CH5N/c1-2-3/h2H,1H3/i1D3,2D","InChI=1S/CH4N/c1-2/h2H,1H3/i1D3,2D",True,"InChI=1S/CH5N/c1-2/h2H2,1H3/i1D3/hD",0.737879,4,9006_1103+1106+1109,/work/input/bms-molecular-translation/train/f/...
3878701,ff46160b9517,"InChI=1S/C2H5N/c1-3-2/h3H,1H3/i1D3,2D","InChI=1S/CH4N/c1-2/h2H,1H3/i1D3,2D",False,"InChI=1S/C2H5N/c1-3-2/h3H,1H3/i1D3,2D",0.095276,5,9006_1103+1106+1109,/work/input/bms-molecular-translation/train/f/...
3878702,ff46160b9517,"InChI=1S/C2H5N/c1-3-2/h3H,1H3/i1D3,2D2","InChI=1S/CH4N/c1-2/h2H,1H3/i1D3,2D",False,"InChI=1S/C2H5N/c1-3-2/h3H,1H3/i1D3,2D2",0.167419,6,9006_1103+1106+1109,/work/input/bms-molecular-translation/train/f/...


In [9]:
MODELs = [
    "1113_swin_large_bert_384",
    "1109_vtnt_bert_512-1024-denoise-5",
]
for model in tqdm(MODELs):
    candidate_df = valid_df.drop_duplicates(subset=["image_id", "normed_InChI"]).query("model != @model")
    candidate_df.to_csv(f"/work/output/{model}/candidate_0521.csv", index=False)